In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 100)

from data_prep_functions import *

In [283]:
name = "EPL_17_18"
base = pd.read_csv("./EPL_data_sofa_score/game_data_" + name + ".csv", sep=";")

In [284]:
base["data"] = pd.to_datetime(base['data'], format="%Y-%m-%d")

## Preparação das Variáveis

In [285]:
def str_percentage_to_float(x):
    x = x.replace("%", "")
    return(float(x)/100)


def split_statistics(base, coluna, new_name, drop_old=True):    
    temp = [x.split(" ")[0] if type(x) == "str" else "0" for x in base[coluna + "_away"]]
    
    # Se tem /, então divide em duas colunas
    if "/" in temp[0]:
        base["Attempted_" + new_name + "_away"] = [int(x.split("/")[1]) for x in temp]
        base["Completed_" + new_name + "_away"] = [int(x.split("/")[0]) for x in temp]
    else:
    # Apenas o total
        base["Total_" + new_name + "_away"] = temp
        base["Total_" + new_name + "_away"] = base["Total_" + new_name + "_away"].astype(int)
    
    base[new_name + "_accuracy_away"] = [x.split(" ")[1].replace("(", "").replace(")", "") 
                                         if type(x) == "str" else "0"
                                         for x in base[coluna + "_away"]]
    base[new_name + "_accuracy_away"] = base[new_name + "_accuracy_away"].apply(str_percentage_to_float)
    
    temp = [x.split(" ")[0] if type(x) == "str" else "0" for x in base[coluna + "_home"]]    
    if "/" in temp[0]:
        base["Attempted_" + new_name + "_home"] = [int(x.split("/")[1]) for x in temp]
        base["Completed_" + new_name + "_home"] = [int(x.split("/")[0]) for x in temp]
    else:
        base["Total_" + new_name + "_home"] = temp
        base["Total_" + new_name + "_home"] = base["Total_" + new_name + "_home"].astype(int)
    
    base[new_name + "_accuracy_home"] = [x.split(" ")[1].replace("(", "").replace(")", "") 
                                         if type(x) == "str" else "0"
                                         for x in base[coluna + "_home"]]
    base[new_name + "_accuracy_home"] = base[new_name + "_accuracy_home"].apply(str_percentage_to_float)
    
    if drop_old:
        base.drop([coluna + "_away", coluna + "_home"], axis=1, inplace=True)

In [286]:
split_statistics(base, "Accurate passes", "Passes")
split_statistics(base, "Crosses", "Crosses")
split_statistics(base, "Dribbles", "Dribbles")
split_statistics(base, "Long balls", "Long_balls")
split_statistics(base, "Tackles", "Tackles")

In [287]:
base["Ball possession_away"] = base["Ball possession_away"].apply(str_percentage_to_float)
base["Ball possession_home"] = base["Ball possession_home"].apply(str_percentage_to_float)

In [289]:
base.fillna(0, inplace=True)

## Cria Features de resposta

In [290]:
base["fl_home_win"] = base["result"].apply(lambda x: (x == 1) * 1)
base["fl_away_win"] = base["result"].apply(lambda x: (x == -1) * 1)
base["fl_draw"] = base["result"].apply(lambda x: (x == 0) * 1)

## Cria Features de dias

In [291]:
base["DAYS_FROM_LAST_GAME_home"] = [get_days_from_last_game(base, x.data, x.team_home) 
                                    for x in base.itertuples()]
base["DAYS_FROM_LAST_GAME_away"] = [get_days_from_last_game(base, x.data, x.team_away) 
                                    for x in base.itertuples()]

## Features baseadas na pressão por minuto

In [332]:
def pos_neg_counts(a):
    mask = a > 0
    idx = np.flatnonzero(mask[1:] != mask[:-1])
    try:
        count = np.concatenate(( [idx[0]+1], idx[1:] - idx[:-1], [a.size-1-idx[-1]] ))
        if a[0]<0:
            return count[1::2], count[::2] # pos, neg counts
        else:
            return count[::2], count[1::2] # pos, neg counts
    except IndexError:
        return np.array([92]), np.array([0])

In [335]:
def cria_features_form_minute(base, i):
    r = {}
    
    cols = [x for x in base.columns if "form_minute" in x]
    linha = base.iloc[i][cols]
    
    # ----------
    # Dominance 
    # ----------
    # Home
    home_dom = linha[linha > 0]
    r["minutes_dominant_home"] = len(home_dom)
    r["total_dominance_home"] = home_dom.sum()
    r["avg_dominance_home"] = home_dom.mean()
    r["max_dominance_home"] = home_dom.max()
    r["min_dominance_home"] = home_dom.min()
    r["std_dominance_home"] = home_dom.std()
    
    # Away
    away_dom = linha[linha < 0] * -1
    r["minutes_dominant_away"] = len(away_dom)
    r["total_dominance_away"] = away_dom.sum()
    r["avg_dominance_away"] = away_dom.mean()
    r["max_dominance_away"] = away_dom.max()
    r["min_dominance_away"] = away_dom.min()
    r["std_dominance_away"] = away_dom.std()
    
    # -----------------
    # Minutes Sequence
    # -----------------
    pos_counts, neg_counts = pos_neg_counts(np.array(linha))
    # Home
    r["max_minutes_sequence_dominant_home"] = max(pos_counts)
    r["std_minutes_sequence_dominant_home"] = np.std(pos_counts)
    
    # Away
    r["max_minutes_sequence_dominant_away"] = max(neg_counts)
    r["std_minutes_sequence_dominant_away"] = np.std(neg_counts)
    
    # ----------------
    # Comparing Teams
    # ----------------
    r["minutes_draw"] = len(linha[linha == 0])
    
    return(r)

In [336]:
temp = []
for i in tqdm_notebook(range(len(base))):
    temp.append(cria_features_form_minute(base, i))
    
temp = pd.DataFrame.from_dict(temp)
base = pd.concat([base, temp], axis=1)

## Estatísticas Médias Últimos 5 Jogos

In [337]:
home_columns = [x for x in base.columns if "_home" in x]
away_columns = [x for x in base.columns if "_away" in x]

In [338]:
data_ref = base["data"].iloc[80]

In [339]:
last_games = get_last_games(base, data_ref, "LIV", n = 5, filter="all", verbose=False)

In [340]:
last_games

,game,Aerials won_away,Aerials won_home,Ball possession_away,Ball possession_home,Big chances missed_away,Big chances missed_home,Big chances_away,Big chances_home,Blocked shots_away,Blocked shots_home,Clearances_away,Clearances_home,Corner kicks_away,Corner kicks_home,Counter attacks_away,Counter attacks_home,Dispossessed_away,Dispossessed_home,Duels won_away,Duels won_home,Fouls_away,Fouls_home,Goalkeeper saves_away,Goalkeeper saves_home,Hit woodwork_away,Hit woodwork_home,Interceptions_away,Interceptions_home,Offsides_away,Offsides_home,Passes_away,Passes_home,Red cards_away,Red cards_home,Shots inside box_away,Shots inside box_home,Shots off target_away,Shots off target_home,Shots on target_away,Shots on target_home,Shots outside box_away,Shots outside box_home,Total shots_away,Total shots_home,Yellow cards_away,Yellow cards_home,away_score,data,form_minute_1,...,form_minute_9,form_minute_90,form_minute_90.5,home_score,hora,result,team_away,team_home,Total_Passes_away,Passes_accuracy_away,Total_Passes_home,Passes_accuracy_home,Total_Crosses_away,Crosses_accuracy_away,Total_Crosses_home,Crosses_accuracy_home,Total_Dribbles_away,Dribbles_accuracy_away,Total_Dribbles_home,Dribbles_accuracy_home,Total_Long_balls_away,Long_balls_accuracy_away,Total_Long_balls_home,Long_balls_accuracy_home,Total_Tackles_away,Tackles_accuracy_away,Total_Tackles_home,Tackles_accuracy_home,fl_home_win,fl_away_win,fl_draw,DAYS_FROM_LAST_GAME_home,DAYS_FROM_LAST_GAME_away,avg_dominance_away,avg_dominance_home,max_dominance_away,max_dominance_home,max_minutes_sequence_dominant_away,max_minutes_sequence_dominant_home,min_dominance_away,min_dominance_home,minutes_dominant_away,minutes_dominant_home,minutes_draw,std_dominance_away,std_dominance_home,std_minutes_sequence_dominant_away,std_minutes_sequence_dominant_home,total_dominance_away,total_dominance_home
30,MCI X LIV 2017-09-09,4,9,0.34,0.66,0.0,0.0,0.0,0.0,2,0,0.0,0.0,3,8,0.0,0.0,0.0,0.0,39,36,9,10,6.0,3.0,0.0,0.0,0.0,0.0,2.0,5.0,372,730,1.0,0.0,2,10,2,3,3,10,5,3,7,13,2.0,2.0,0,2017-09-09,-10,...,68,5.0,80.0,5,09:30:00,1,LIV,MCI,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0,0,14.0,13.0,17.736842,28.095890,55.0,105.0,8,20,1.0,3.0,19,73,0,15.494953,24.042881,2.182987,5.586580,337.0,2051.0
43,LIV X BUR 2017-09-16,15,8,0.29,0.71,0.0,0.0,0.0,0.0,0,12,0.0,0.0,2,12,0.0,0.0,0.0,0.0,35,36,9,7,8.0,2.0,0.0,1.0,0.0,0.0,5.0,1.0,277,676,0.0,0.0,4,18,1,14,4,9,1,17,5,35,2.0,1.0,1,2017-09-16,10,...,22,59.0,65.0,1,12:00:00,0,BUR,LIV,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1,7.0,6.0,33.727273,31.308642,80.0,100.0,4,38,3.0,5.0,11,81,0,28.003896,21.592616,1.166190,12.446552,371.0,2536.0
57,LEI X LIV 2017-09-23,30,24,0.57,0.43,0.0,0.0,0.0,0.0,8,3,0.0,0.0,6,3,2.0,0.0,0.0,0.0,68,61,14,13,5.0,3.0,1.0,0.0,0.0,0.0,1.0,4.0,486,360,0.0,0.0,13,9,9,2,6,7,10,3,23,12,3.0,3.0,3,2017-09-23,-7,...,-18,-2.0,-5.0,2,14:30:00,-1,LIV,LEI,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1,0,7.0,7.0,26.764706,30.833333,85.0,105.0,16,7,1.0,1.0,68,24,0,22.535622,30.336757,4.135286,1.683251,1820.0,740.0
69,NEW X LIV 2017-10-01,20,17,0.68,0.32,0.0,0.0,0.0,0.0,6,2,0.0,0.0,5,1,0.0,0.0,0.0,0.0,57,39,6,11,4.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,613,295,0.0,0.0,10,2,9,1,2,5,7,6,17,8,1.0,1.0,1,2017-10-01,10,...,28,11.0,30.0,1,13:30:00,0,LIV,NEW,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1,7.0,8.0,21.323529,16.869565,75.0,72.0,10,7,1.0,1.0,68,23,1,16.662982,17.022190,3.249921,1.705791,1450.0,388.0
70,LIV X MUN 2017-10-14,11,11,0.38,0.62,0.0,0.0,0.0,0.0,3,6,0.0,0.0,3,7,0.0,0.0,0.0,0.0,49,63,13,7,5.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,354,580,0.0,0.0,2,11,2,8,1,5,4,8,6,19,2.0,0.0,0,2017-10-14,-4,...,5,6.0,100.0,0,09:30:00,0,MUN,LIV,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1,13.0,14.0,13.846154,22.646154,59.0,100.0,8,22,2.0,2.0,26,65,1,14.161052,17.572556,2.061052,5.853585,360.0,1472.0


In [341]:
get_avg_last_games(last_games, "LIV", home_columns, away_columns,
                       n = 5, data_ref = data_ref, rivals = False, 
                       to_drop=["fl_win", "Total_passes", "result", "Accurate passes", "hora", "game"])

,Aerials won,Ball possession,Big chances missed,Big chances,Blocked shots,Clearances,Corner kicks,Counter attacks,Dispossessed,Duels won,Fouls,Goalkeeper saves,Hit woodwork,Interceptions,Offsides,Passes,Red cards,Shots inside box,Shots off target,Shots on target,Shots outside box,Total shots,Yellow cards,away_score,form_minute_1,form_minute_10,form_minute_11,form_minute_12,form_minute_13,form_minute_14,form_minute_15,form_minute_16,form_minute_17,form_minute_18,form_minute_19,form_minute_2,form_minute_20,form_minute_21,form_minute_22,form_minute_23,form_minute_24,form_minute_25,form_minute_26,form_minute_27,form_minute_28,form_minute_29,form_minute_3,form_minute_30,form_minute_31,form_minute_32,...,home_score,Total_Passes,Passes_accuracy,Total_Crosses,Crosses_accuracy,Total_Dribbles,Dribbles_accuracy,Total_Long_balls,Long_balls_accuracy,Total_Tackles,Tackles_accuracy,fl_win,fl_draw,DAYS_FROM_LAST_GAME,avg_dominance,max_dominance,max_minutes_sequence_dominant,min_dominance,minutes_dominant,minutes_draw,std_dominance,std_minutes_sequence_dominant,total_dominance,N_WINS_HOME,N_GAMES_HOME,WIN_HOME_PCT,N_WINS_AWAY,N_GAMES_AWAY,WIN_AWAY_PCT,N_WINS_TOTAL,WIN_PCT,TOTAL_DAYS_DIFF,DAYS_DIFF_LG_STD,DAYS_DIFF_LG_MEAN,N_GAMES_L2_days,N_GAMES_L4_days,N_GAMES_L6_days,N_GAMES_L8_days,N_GAMES_L10_days,N_GAMES_AWAY_L2_days,N_GAMES_AWAY_L4_days,N_GAMES_AWAY_L6_days,N_GAMES_AWAY_L8_days,N_GAMES_AWAY_L10_days,AVG_DAYS_FROM_LG,STD_DAYS_FROM_LG,MIN_DAYS_FROM_LG,total_minutes_dominant,avg_total_minutes_dominant,avg_total_dominance
0,14.6,0.584,0.0,0.0,6.8,0.0,6.6,0.4,0.0,52.6,8.6,3.6,0.6,0.0,1.8,545.4,0.2,10.8,8.4,5.0,9.4,20.2,1.4,1.0,-0.2,23.4,18.6,10.6,29.4,12.4,5.4,-2.0,-0.6,-14.8,3.2,-6.2,-5.4,13.6,11.6,9.4,-19.4,2.2,-7.8,-8.2,-18.2,-4.2,-9.6,-20.4,6.8,-1.4,...,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,9.6,23.955975,83.0,18.8,2.0,60.2,0.4,18.771746,5.573666,1523.0,2.0,2.0,1.0,2.0,3.0,0.666667,4.0,0.8,35.0,2.481935,8.2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,9.6,2.8,7.0,301.0,60.2,1523.0


## Monta a base final

In [342]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [343]:
base_final = gera_last_N_games(base, all_games = base, N = [5],
                              to_drop=['fl_home_win', 'game', 'hora', 'result', 'data', 'index',
                                       'fl_away_win', 'fl_draw', 'fl_win_home'])

In [344]:
base_final.to_csv("EPL_data_sofa_score/Processadas/" + name + "_per_game.csv", sep=";")

In [345]:
base_final_delta_cross = variaveis_delta(
    base_final, to_predict=False,
    keep_features=['team_home', 'team_away', 'home_score', 'away_score', 'data', 'hora', 'result',
                  'fl_home_win', 'fl_away_win', 'fl_draw']
)

In [346]:
base_final_delta_cross.to_csv("EPL_data_sofa_score/Processadas/" + name + "_delta_cross.csv", sep=";")

In [347]:
base_final_delta_cross

,team_home,team_away,home_score,away_score,data,hora,result,fl_home_win,fl_away_win,fl_draw,fl_home_win,D1_form_minute_3_L5,D2_form_minute_3_L5,D1_form_minute_39_L5,D2_form_minute_39_L5,D1_form_minute_74_L5,D2_form_minute_74_L5,D1_N_GAMES_AWAY_L2_days_L5,D2_N_GAMES_AWAY_L2_days_L5,D1_total_dominance_L5,D2_total_dominance_L5,D1_form_minute_8_L5,D2_form_minute_8_L5,D1_form_minute_53_L5,D2_form_minute_53_L5,D1_form_minute_63_L5,D2_form_minute_63_L5,D1_form_minute_32_L5,D2_form_minute_32_L5,D1_N_GAMES_L8_days_L5,D2_N_GAMES_L8_days_L5,D1_Yellow cards_L5,D2_Yellow cards_L5,D1_form_minute_14_L5,D2_form_minute_14_L5,D1_Total_Passes_L5,D2_Total_Passes_L5,D1_DAYS_FROM_LAST_GAME_L5,D2_DAYS_FROM_LAST_GAME_L5,D1_form_minute_65_L5,D2_form_minute_65_L5,D1_form_minute_9_L5,D2_form_minute_9_L5,D1_N_GAMES_L4_days_L5,D2_N_GAMES_L4_days_L5,D1_form_minute_57_L5,D2_form_minute_57_L5,D1_Dribbles_accuracy_L5,D2_Dribbles_accuracy_L5,D1_form_minute_73_L5,...,D1_N_GAMES_AWAY_L5,D2_N_GAMES_AWAY_L5,D1_form_minute_42_L5,D2_form_minute_42_L5,D1_max_minutes_sequence_dominant_L5,D2_max_minutes_sequence_dominant_L5,D1_form_minute_18_L5,D2_form_minute_18_L5,D1_Total_Dribbles_L5,D2_Total_Dribbles_L5,D1_form_minute_54_L5,D2_form_minute_54_L5,D1_Long_balls_accuracy_L5,D2_Long_balls_accuracy_L5,D1_total_minutes_dominant_L5,D2_total_minutes_dominant_L5,D1_form_minute_69_L5,D2_form_minute_69_L5,D1_DAYS_DIFF_LG_STD_L5,D2_DAYS_DIFF_LG_STD_L5,D1_Duels won_L5,D2_Duels won_L5,D1_Total shots_L5,D2_Total shots_L5,D1_Total_Tackles_L5,D2_Total_Tackles_L5,D1_Big chances missed_L5,D2_Big chances missed_L5,D1_form_minute_28_L5,D2_form_minute_28_L5,D1_form_minute_30_L5,D2_form_minute_30_L5,D1_form_minute_43_L5,D2_form_minute_43_L5,D1_form_minute_59_L5,D2_form_minute_59_L5,D1_form_minute_78_L5,D2_form_minute_78_L5,D1_min_dominance_L5,D2_min_dominance_L5,D1_N_GAMES_HOME_L5,D2_N_GAMES_HOME_L5,D1_form_minute_5_L5,D2_form_minute_5_L5,D1_minutes_draw_L5,D2_minutes_draw_L5,D1_form_minute_90_L5,D2_form_minute_90_L5,D1_form_minute_80_L5,D2_form_minute_80_L5
ARS X LEI 2017-08-11,ARS,LEI,4,3,2017-08-11 00:00:00,16:45:00,1,1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WAT X LIV 2017-08-12,WAT,LIV,3,3,2017-08-12 00:00:00,09:30:00,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHE X BUR 2017-08-12,CHE,BUR,2,3,2017-08-12 00:00:00,12:00:00,-1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CRY X HUD 2017-08-12,CRY,HUD,0,3,2017-08-12 00:00:00,12:00:00,-1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EVE X STO 2017-08-12,EVE,STO,1,0,2017-08-12 00:00:00,12:00:00,1,1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0